In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

In [22]:
from src.utils.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from pickle import dump, load
import pandas as pd
import numpy as np

from src.bias.BiasDetector import BiasDetector
from src.bias.TotalVariationDistance import TotalVariationDistance
from src.bias.JSDivergence import JSDivergence

## Importing Data and Trained Classifier

In [4]:
input_data_path = "../data/raw_data/uci-default-of-credit-card/data/data.csv"
local_path_save = '../data/mlflow_artifacts/'

In [5]:
fitted_ohe = load(open(local_path_save + '_ohe.pkl', 'rb')) 
fitted_scaler = load(open(local_path_save + '_scaler.pkl', 'rb'))

/Users/davideposillipo/.pyenv/versions/3.10.9/envs/prova/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/davideposillipo/.pyenv/versions/3.10.9/envs/prova/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
pp = Preprocessing(input_data_path, "default")
X, Y = pp.read_dataframe()

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=420)

X_test_ohe, _, _ = pp.preprocess_for_classification(df=X_test, 
                                                fit_ohe=True, 
                                                fitted_ohe=fitted_ohe,
                                                perform_scaling=True,
                                                fitted_scaler=fitted_scaler)

In [7]:
with open("trained_model_for_testing/RF_12_200.pkl", "rb") as file:
    classifier_strong = load(file)
    
with open("trained_model_for_testing/RF_37_10.pkl", "rb") as file:
    classifier_weak = load(file)
    
with open("trained_model_for_testing/Tree_depth2.pkl", "rb") as file:
    classifier_lame = load(file)

In [8]:
predicted_values_strong = classifier_strong.predict(X_test_ohe)
predicted_values_weak = classifier_weak.predict(X_test_ohe)
predicted_values_lame = classifier_lame.predict(X_test_ohe)

In [9]:
def age_buckets(x):
    if x < 30:
        return 1
    elif x < 40:
        return 2
    else:
        return 3

X_test['age_buckets'] = X.x5_age.apply(age_buckets)

In [10]:
conditioning_variables = ['x3_education', 'x4_marriage', 'age_buckets']

In [11]:
df_with_predictions_strong = pd.concat(
    [X_test.reset_index(drop=True), pd.Series(predicted_values_strong)], axis=1).rename(columns={0:"predictions"})

df_with_predictions_weak = pd.concat(
    [X_test.reset_index(drop=True), pd.Series(predicted_values_weak)], axis=1).rename(columns={0:"predictions"})

df_with_predictions_lame = pd.concat(
    [X_test.reset_index(drop=True), pd.Series(predicted_values_lame)], axis=1).rename(columns={0:"predictions"})

## Experiment 1: freqs-vs-freqs, TVD, A1=low, root_variable=x2_sex

In [12]:
d = TotalVariationDistance()
bd = BiasDetector(distance=d, A1="low")

#### Strong model, no conditioning variables

In [13]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.025269625352224545, True, 0.038868585412256317)

#### Weak model, no conditioning variables

In [14]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.02310418899075288, True, 0.038868585412256317)

#### Lame model, no conditioning variables

In [15]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.011162043100369085, True, 0.038868585412256317)

## Experiment 2: freqs-vs-freqs, TVD, A1=low, root_variable=x3_education

In [17]:
d = TotalVariationDistance()
bd = BiasDetector(distance=d, A1="low")

#### Strong model, no conditioning variables

In [18]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.14609739826551038, False, 0.030917574992439394)

#### Weak model, no conditioning variables

In [19]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.1428571428571429, False, 0.030917574992439394)

#### Lame model, no conditioning variables

In [20]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.1428571428571429, False, 0.030917574992439394)

## Experiment 3: freqs-vs-freqs, JS, A1=low, root_variable=x2_sex

In [23]:
d = JSDivergence()
bd = BiasDetector(distance=d, A1="low")

#### Strong model, no conditioning variables

In [24]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.03382573454226581, True, 0.038868585412256317)

#### Weak model, no conditioning variables

In [25]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.030599211687988202, True, 0.038868585412256317)

#### Lame model, no conditioning variables

In [26]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.017047162918550674, True, 0.038868585412256317)

## Experiment 4: freqs-vs-freqs, JS, A1=low, root_variable=x3_education

In [27]:
d = JSDivergence()
bd = BiasDetector(distance=d, A1="low")

#### Strong model, no conditioning variables

In [28]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.27785884217449525, False, 0.030917574992439394)

#### Weak model, no conditioning variables

In [29]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.2745832066054148, False, 0.030917574992439394)

#### Lame model, no conditioning variables

In [30]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.2745832066054148, False, 0.030917574992439394)

## Experiment 5: freqs-vs-freqs, JS, A1=low, root_variable=x6_pay_0 (discrete)

In [39]:
df_with_predictions_strong.x6_pay_0.

 0    4415
-1    1726
 1    1076
-2     851
 2     788
 3      98
 4      27
 8       6
 5       6
 6       4
 7       3
Name: x6_pay_0, dtype: int64

In [46]:
def discrete_x6(x):
    if x <= 2:
        return x
    else:
        return 3

df_with_predictions_lame["x6_pay_0_discrete"] = df_with_predictions_lame["x6_pay_0"].apply(lambda x: discrete_x6(x))
df_with_predictions_strong["x6_pay_0_discrete"] = df_with_predictions_strong["x6_pay_0"].apply(lambda x: discrete_x6(x))
df_with_predictions_weak["x6_pay_0_discrete"] = df_with_predictions_weak["x6_pay_0"].apply(lambda x: discrete_x6(x))

In [47]:
d = JSDivergence()
bd = BiasDetector(distance=d, A1="low")

#### Strong model, no conditioning variables

In [48]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x6_pay_0_discrete'
)

(0.8781746071094385, False, 0.03144764235376052)

#### Weak model, no conditioning variables

In [49]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x6_pay_0_discrete'
)

(0.7063288037442578, False, 0.03144764235376052)

#### Lame model, no conditioning variables

In [50]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x6_pay_0_discrete'
)

(0.9267799977583331, False, 0.03144764235376052)

## Experiment 6: freqs-vs-freqs, JS, A1=low, root_variable=x2_sex, conditioning_variable=x6_pay (discrete)

In [47]:
d = JSDivergence()
bd = BiasDetector(distance=d, A1="low")

#### Strong model

In [53]:
results_strong = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [54]:
results_strong

{'x6_pay_0_discrete==0': (4415, 0.005796418752555453, True),
 'x6_pay_0_discrete==2': (788, 0.00026061707568029194, True),
 'x6_pay_0_discrete==1': (1076, 0.1039149975268626, False),
 'x6_pay_0_discrete==-2': (851, 0.0, True),
 'x6_pay_0_discrete==-1': (1726, 0.005827564760104745, True),
 'x6_pay_0_discrete==3': (144, 0.02091711052564077, True)}

#### Weak model

In [55]:
results_weak = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [56]:
results_weak

{'x6_pay_0_discrete==0': (4415, 0.015088851952557734, True),
 'x6_pay_0_discrete==2': (788, 0.013215421946293956, True),
 'x6_pay_0_discrete==1': (1076, 0.07499466492030307, False),
 'x6_pay_0_discrete==-2': (851, 0.0746013936850334, False),
 'x6_pay_0_discrete==-1': (1726, 0.015667497429594533, True),
 'x6_pay_0_discrete==3': (144, 0.1032652540334566, False)}

#### Lame model

In [57]:
results_lame = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [58]:
results_lame

{'x6_pay_0_discrete==0': (4415, 0.0, True),
 'x6_pay_0_discrete==2': (788, 0.011942882267781, True),
 'x6_pay_0_discrete==1': (1076, 0.0, True),
 'x6_pay_0_discrete==-2': (851, 0.0, True),
 'x6_pay_0_discrete==-1': (1726, 0.0, True),
 'x6_pay_0_discrete==3': (144, 0.0, True)}

## Experiment 7: freqs-vs-freqs, TVD, A1=low, root_variable=x2_sex, conditioning_variable=x6_pay (discrete)

In [59]:
d = TotalVariationDistance()
bd = BiasDetector(distance=d, A1="low")

#### Strong model

In [60]:
results_strong = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [61]:
results_strong

{'x6_pay_0_discrete==0': (4415, 0.0005099035448375608, True),
 'x6_pay_0_discrete==2': (788, 0.00017791835524372246, True),
 'x6_pay_0_discrete==1': (1076, 0.08358658256880735, False),
 'x6_pay_0_discrete==-2': (851, 0.0, True),
 'x6_pay_0_discrete==-1': (1726, 0.001151020781208943, True),
 'x6_pay_0_discrete==3': (144, 0.01550688117852296, True)}

#### Weak model

In [62]:
results_weak = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [63]:
results_weak

{'x6_pay_0_discrete==0': (4415, 0.004162196434452259, True),
 'x6_pay_0_discrete==2': (788, 0.013502026292379177, True),
 'x6_pay_0_discrete==1': (1076, 0.07061353211009175, False),
 'x6_pay_0_discrete==-2': (851, 0.030280300039478902, True),
 'x6_pay_0_discrete==-1': (1726, 0.005857924620991439, True),
 'x6_pay_0_discrete==3': (144, 0.11145570847063391, False)}

#### Lame model

In [64]:
results_lame = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [65]:
results_lame

{'x6_pay_0_discrete==0': (4415, 0.0, True),
 'x6_pay_0_discrete==2': (788, 0.005996507528582273, True),
 'x6_pay_0_discrete==1': (1076, 0.0, True),
 'x6_pay_0_discrete==-2': (851, 0.0, True),
 'x6_pay_0_discrete==-1': (1726, 0.0, True),
 'x6_pay_0_discrete==3': (144, 0.0, True)}